In [1]:
import wbgapi as wb
import pandas as pd
import os

# 1. Configuração dos Indicadores
# Selecionamos métricas que conversam com Governança e Economia
indicadores = {
    'NY.GDP.MKTP.CD': 'PIB_USD',           # Tamanho da economia
    'NY.GDP.PCAP.CD': 'PIB_per_Capita',    # Riqueza média (melhor que PIB total para comparar corrupção)
    'SP.POP.TOTL': 'Populacao',            # Para normalizar dados
    'IQ.CPA.PROP.XQ': 'Rating_Comercio'    # Índice de facilidade de comércio (proxy de burocracia)
}

# 2. Extração (Extract)
print("⏳ Iniciando download do Banco Mundial via API...")

# time=range(2014, 2024) -> Pega os últimos 10 anos para análise temporal
df_wb = wb.data.DataFrame(
    indicadores.keys(), 
    labels=True, 
    time=range(2014, 2024), 
    skipAggs=True  # Ignora continentes, pega só países
)

# 3. Transformação Básica (Transform)
# O wbgapi retorna o índice como o código do país (ex: ZWE). Vamos mover para coluna.
df_wb = df_wb.reset_index()

# 4. Salvando os dados (Load)
# Verifica se a pasta datasets existe, se não, cria
os.makedirs('../datasets', exist_ok=True)

caminho_arquivo = '../datasets/world_bank_raw.csv'
df_wb.to_csv(caminho_arquivo, index=False)

print(f"✅ Sucesso! Arquivo salvo em: {caminho_arquivo}")
print(f"📊 Dimensões dos dados: {df_wb.shape}")
df_wb.head()

c:\Users\gusta\OneDrive\Área de Trabalho\projeto_cpi_economy\.venv\Lib\site-packages\wbgapi\__init__.py:639: SyntaxWarning: invalid escape sequence '\w'
  pattern = '(?<!\w).{{0,{len}}}{term}.{{0,{len}}}(?!\w)'.format(term=re.escape(q), len=padding)


⏳ Iniciando download do Banco Mundial via API...
✅ Sucesso! Arquivo salvo em: ../datasets/world_bank_raw.csv
📊 Dimensões dos dados: (868, 14)


,economy,series,Country,Series,YR2014,YR2015,YR2016,YR2017,YR2018,YR2019,YR2020,YR2021,YR2022,YR2023
0,ZWE,NY.GDP.MKTP.CD,Zimbabwe,GDP (current US$),1.949555e+10,1.996306e+10,2.054876e+10,5.107473e+10,3.415606e+10,2.571566e+10,2.686856e+10,2.724051e+10,3.278966e+10,3.523137e+10
1,ZMB,NY.GDP.MKTP.CD,Zambia,GDP (current US$),2.714102e+10,2.125122e+10,2.095841e+10,2.587360e+10,2.631151e+10,2.330867e+10,1.813776e+10,2.209642e+10,2.916378e+10,2.757796e+10
2,YEM,NY.GDP.MKTP.CD,"Yemen, Rep.",GDP (current US$),4.322859e+10,4.244449e+10,3.131783e+10,2.684223e+10,2.160616e+10,NaN,NaN,NaN,NaN,NaN
3,PSE,NY.GDP.MKTP.CD,West Bank and Gaza,GDP (current US$),1.398970e+10,1.397240e+10,1.540540e+10,1.612800e+10,1.627660e+10,1.713350e+10,1.553170e+10,1.810900e+10,1.916550e+10,1.784790e+10
4,VIR,NY.GDP.MKTP.CD,Virgin Islands (U.S.),GDP (current US$),3.565000e+09,3.663000e+09,3.798000e+09,3.794000e+09,3.923000e+09,4.121000e+09,4.229000e+09,4.507000e+09,4.672000e+09,NaN


In [3]:
# --- ETAPA DE LIMPEZA E TRANSFORMAÇÃO (CORRIGIDA) ---

# 1. Melt: Adicionamos 'Country' e 'Series' nos identificadores
# Isso impede que o pandas tente transformar o nome do país em um ano
df_melted = df_wb.melt(id_vars=['economy', 'series', 'Country', 'Series'], 
                       var_name='ano', 
                       value_name='valor')

# 2. Limpeza da coluna Ano
# Agora sim, só sobraram os anos (YR2014...) na coluna 'ano'
df_melted['ano'] = df_melted['ano'].str.replace('YR', '').astype(int)

# 3. Pivot: Incluímos 'Country' no índice para manter o nome do país no final
df_cleaned = df_melted.pivot_table(index=['economy', 'Country', 'ano'], 
                                   columns='series', 
                                   values='valor').reset_index()

# 4. Renomear as colunas
# O pivot usa os códigos (NY.GDP...), então renomeamos para ficar bonito
df_cleaned = df_cleaned.rename(columns=indicadores)

# Removemos o nome das colunas do índice (estético)
df_cleaned.columns.name = None

# 5. Salvar
df_cleaned.to_csv('../datasets/world_bank_cleaned.csv', index=False)

print("✨ Dados transformados com sucesso!")
print(df_cleaned.head())

✨ Dados transformados com sucesso!
  economy Country   ano  Rating_Comercio       PIB_USD  PIB_per_Capita  \
0     ABW   Aruba  2014              NaN  2.790850e+09    26129.839062   
1     ABW   Aruba  2015              NaN  2.962907e+09    27458.225331   
2     ABW   Aruba  2016              NaN  2.983635e+09    27441.529662   
3     ABW   Aruba  2017              NaN  3.092429e+09    28440.051964   
4     ABW   Aruba  2018              NaN  3.276184e+09    30082.127645   

   Populacao  
0   106807.0  
1   107906.0  
2   108727.0  
3   108735.0  
4   108908.0  


In [8]:
# Vamos ler o arquivo sem pular linhas para ver onde está o cabeçalho
df_debug = pd.read_excel('../datasets/cpi_history.xlsx', sheet_name='CPI Timeseries 2012 - 2023', skiprows=2)

# Mostra as primeiras 5 linhas
print("🔍 Primeiras linhas do arquivo:")
display(df_debug.head())

# Mostra os nomes das colunas que o pandas detectou
print("\n📋 Colunas detectadas:")
print(df_debug.columns.tolist())

🔍 Primeiras linhas do arquivo:


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45
0,Country / Territory,ISO3,Region,CPI score 2023,Rank 2023,Sources 2023,Standard error 2023,CPI score 2022,Rank 2022,Sources 2022,...,Standard error 2015,CPI score 2014,Sources 2014,Standard error 2014,CPI Score 2013,Sources 2013,Standard error 2013,CPI Score 2012,Sources 2012,Standard error 2012
1,Afghanistan,AFG,AP,20,162,5,6.244771,24,150,4,...,3.49,12,4,1.29,8,3,3.3,8,3,3.3
2,Albania,ALB,ECA,37,98,7,1.560074,36,101,8,...,3.58,33,7,1.51,31,7,2.1,33,7,2
3,Algeria,DZA,MENA,36,104,7,2.327238,33,116,6,...,1.85,36,5,2.06,36,6,2.8,34,6,3.3
4,Angola,AGO,SSA,33,121,7,2.813869,33,116,7,...,1.8,19,6,1.71,23,7,2.8,22,7,1.8



📋 Colunas detectadas:
['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45']


In [9]:
# --- FASE 2: INGESTÃO E MERGE DO CPI (Versão Corrigida) ---

# Ajuste aqui se o nome da aba mudar. Pelo seu print, é este mesmo:
NOME_DA_ABA = 'CPI Timeseries 2012 - 2023' 

print(f"📂 Lendo aba '{NOME_DA_ABA}'...")

# 1. Carregar o Excel com o cabeçalho certo
# MUDANÇA: Usamos header=3 para pegar a linha onde diz "Country", "ISO3", etc.
df_cpi_raw = pd.read_excel('../datasets/cpi_history.xlsx', sheet_name=NOME_DA_ABA, header=3)

# 2. Limpeza (Data Cleaning)
# Agora o 'ISO3' vai ser encontrado corretamente nas colunas
cols_to_keep = ['ISO3'] + [col for col in df_cpi_raw.columns if 'CPI score' in str(col)]
df_cpi = df_cpi_raw[cols_to_keep].copy()

# Renomear colunas para deixar só o ano (ex: "CPI score 2023" -> "2023")
df_cpi.columns = [col.replace('CPI score ', '') for col in df_cpi.columns]

# 3. Transformação (Wide to Long)
df_cpi_melted = df_cpi.melt(id_vars=['ISO3'], var_name='ano', value_name='CPI_Score')

# Tratamento de tipos
df_cpi_melted['ano'] = df_cpi_melted['ano'].astype(int)
df_cpi_melted['ISO3'] = df_cpi_melted['ISO3'].astype(str).str.strip()

# 4. Merge com dados do Banco Mundial (df_cleaned)
print("🔄 Cruzando dados de Economia com Corrupção...")
df_final = pd.merge(df_cleaned, df_cpi_melted, 
                    left_on=['economy', 'ano'], 
                    right_on=['ISO3', 'ano'], 
                    how='inner')

# 5. Salvar o Dataset Mestre
df_final.to_csv('../datasets/master_dataset.csv', index=False)

print(f"✅ SUCESSO! Dataset Mestre criado com {df_final.shape[0]} linhas.")
print(df_final.head())

📂 Lendo aba 'CPI Timeseries 2012 - 2023'...
🔄 Cruzando dados de Economia com Corrupção...
✅ SUCESSO! Dataset Mestre criado com 1790 linhas.
  economy      Country   ano  Rating_Comercio       PIB_USD  PIB_per_Capita  \
0     AFG  Afghanistan  2014              1.5  2.049713e+10      625.054942   
1     AFG  Afghanistan  2015              1.5  1.913422e+10      565.569730   
2     AFG  Afghanistan  2016              2.0  1.811657e+10      522.082216   
3     AFG  Afghanistan  2017              2.0  1.875346e+10      525.469771   
4     AFG  Afghanistan  2018              2.0  1.805322e+10      491.337221   

    Populacao ISO3  CPI_Score  
0  32792523.0  AFG       12.0  
1  33831764.0  AFG       11.0  
2  34700612.0  AFG       15.0  
3  35688935.0  AFG       15.0  
4  36743039.0  AFG       16.0  
